In [ ]:
import os
import re
from flask import Flask, request
from slack_sdk import WebClient
from slack_bolt import App
from slack_bolt.adapter.flask import SlackRequestHandler
from dotenv import find_dotenv, load_dotenv
from fredapi import Fred
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# Load environment variables
load_dotenv(find_dotenv())

# Set Slack API credentials and FRED API key
SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]
FRED_API_KEY = os.environ["FRED_API_KEY"]

# Initialize the Slack app and Flask app
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

# Initialize the OpenAI-based language model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create a memory buffer for conversational context
memory = ConversationBufferMemory()

# Create system and human prompts for Fredrick-specific conversations
system_template = """
You are Fredrick, an AI assistant specialized in fetching economic data from FRED.
Your role is to provide economic data on demand and help users understand it.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "User says: {user_input}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Create a conversational chain with memory for FRED-related queries
chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# Initialize the FRED API
fred = Fred(api_key=FRED_API_KEY)

# Function to fetch data from FRED based on a series ID
def fetch_data_from_fred(series_id):
    try:
        series = fred.get_series(series_id)
        latest_value = series.iloc[-1
]
        latest_date = series.index[-1]
        return latest_date, latest_value
    except Exception as e:
        return None, None

# Slack event handler for app mentions
@app.event("app_mention")
def handle_app_mentions(event, say):
    user_text = event.get("text", "").lower()

    # Respond to greetings
    if user_text in ["hi", "hello", "hey"]:
        say("Hello! I'm Fredrick, your AI assistant for economic data. How can I help you today?")
        return

    # Respond to FRED-related queries
    if "gdp" in user_text:
        date, value = fetch_data_from_fred("GDP")
        if date and value:
            say(f"Latest GDP data from FRED: {date} - ${value:,.2f} (in billions)")
        else:
            say("I couldn't fetch the latest GDP data. Please try again later.")
        return

    if "cpi" in user_text:
        date, value = fetch_data_from_fred("CPIAUCSL")
        if date and value:
            say(f"Latest CPI data from FRED: {date} - {value}")
        else:
            say("I couldn't fetch the latest CPI data. Please try again later.")
        return

    if "unemployment rate" in user_text:
        date, value = fetch_data_from_fred("UNRATE")
        if date and value:
            say(f"Latest Unemployment Rate from FRED: {date} - {value}%")
        else:
            say("I couldn't fetch the latest unemployment rate. Please try again later.")
        return

    # Handle general queries with conversational chain
    response = chain.run({"user_input": user_text})
    say(response)

# Flask endpoint for Slack events
@flask_app.route("/slack/events", methods=["POST"])
def slack_events():
    return handler.handle(request)

# Initialize and run the Flask app
if __name__ == "__main__":
    flask_app.run()



C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
